In [1]:
import requests
from itertools import batched
from pathlib import Path
from gis_county import get_tile_id, get_tif_download_urls

In [2]:
WORKING_DIR = Path("Test")
PATH_LIST_TILES = WORKING_DIR/"tile_list"
COOK_COUNTY_GIS_URL = (
    "https://gis.cookcountyil.gov/imagery/rest/services/CookOrtho2023/ImageServer"
)

In [3]:
path_list_ids = WORKING_DIR/"tile_ids"
(download_dir := WORKING_DIR/"download").mkdir(exist_ok=True)

In [4]:
with open(PATH_LIST_TILES) as f:
    tile_list = [f"'{x.strip()}'" for x in f.readlines()]

In [5]:
tile_ids = []

for tile_chunck in batched(tile_list, 20):
    response = get_tile_id(tile_chunck)
    tile_ids += response['objectIds']

In [6]:
# Write to a file the list of tiles ids
with open(path_list_ids, "w") as f:
    for tile_id in tile_ids:
        f.write(str(tile_id) + "\n")

In [15]:
for tile_id in tile_ids:
    for raster_json in get_tif_download_urls(tile_id)['rasterFiles']:
        raster_url = raster_json['id']
        raster_file = download_dir/(raster_url.rpartition('\\')[-1])
        raster_id = raster_json['rasterIds'][0]
        
        # /file?id=.%5C2023%5CRGBNIR%5C14507850.tif&amp;rasterId=3444

        if raster_file.exists():
            pass
            print(f"Skipping {raster_file}, already downloaded")
    
        else:
            print(f"Downloading {raster_file}")

            payload = dict(
                id=raster_url,
                rasterId=raster_id,
            )

            r = requests.get(f"{COOK_COUNTY_GIS_URL}/file?", params=payload, stream=True)
            r.raise_for_status()
        
            with open(raster_file, 'wb') as f:
                f.write(r.content)

    break
        #     # shutil.move(filepath, dst)
        #     # print(f"Downloaded {filename}")

.\2023\RGBNIR\15007750.tif
Test/download/15007750.tif
3640
.\2023\RGBNIR\15007750.tif.ovr
Test/download/15007750.tif.ovr
3640
.\2023\RGBNIR\15007750.tif.aux.xml
Test/download/15007750.tif.aux.xml
3640
.\2023\RGBNIR\15007750.tif.xml
Test/download/15007750.tif.xml
3640
